In [1]:
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logging.basicConfig(
    level=logging.INFO,  # Set the minimum logging level to INFO
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

sys.path.append('../')
from PMpostprocess import PMpostprocess as pp
from PMpostprocess import signal_processing as sp

profilemonitor_plot_util start


In [2]:
import warnings
from contextlib import contextmanager

@contextmanager
def capture_warnings():
    """
    A context manager to capture and print warnings within a block of code.
    """
    with warnings.catch_warnings(record=True) as caught_warnings:
        warnings.simplefilter("always")  # Always capture warnings
        yield  # Allow code execution within the context
        for w in caught_warnings:
            print(f"Warning message: {w.message}")
            print(f"Warning type: {w.category.__name__}")
            print(f"Originated from: {w.filename}, line {w.lineno}")

In [3]:
sys.path.append('../../')
from FRIB_model import ISAAC_helper as ih

FRIB_model version: 1.0.0. updated on 2024-03-05


In [4]:
import os
ISAAC_database_path = "/home/devuser/sf_HWANG/Workspace/BPM4pickup/ISAAC_data_PMver0"
ISAAC_data_rel_path = "20231219_010647_maruta_FS1_CSS_PM_D2225_pv_scan"
#"20240528_200129_hwang_FS1_CSS_PM_D2225_pv_scan"
isac_data = ih.get_most_recent_reconst_data(ISAAC_data_rel_path,ISAAC_database_path)

In [5]:
isac_data.keys()

dict_keys(['reconst_summary', 'reconst_input', 'reconst_output', 'fmlatfile'])

# summary data

In [6]:
isac_data['reconst_summary'].keys()

dict_keys(['comment', 'time', 'jsonfname', 'fname', 'pvinfo', 'scan_type', 'monitorl', 'putPVnamel', 'getPVnamel', 'initputPVvall', 'initgetPVvall', 'scan_data'])

In [7]:
isac_data['reconst_summary']['scan_data'][0].keys()

dict_keys(['putPVvall', 'getPVvall', 'res_monitorl'])

In [8]:
isac_data['reconst_summary']['scan_data'][0]['getPVvall']

{'FS1_CSS:PSQ_D2194:I_RD': 48.865, 'FS1_CSS:PSQ_D2202:I_RD': 159.306}

In [9]:
tmp = isac_data['reconst_summary']['scan_data'][0]['res_monitorl'][0]
tmp['name'], tmp['file'], tmp['coord'], tmp['xrms']

('FS1_CSS:PM_D2225',
 './/FS1_CSS_PM_D2225_20231219_010705.dat',
 'Suxy',
 5.845242977240003)

# reconst input data

In [10]:
isac_data['reconst_input'].keys()

dict_keys(['meas', 'reconst_input', 'jsonfname'])

In [11]:
isac_data['reconst_input']['meas'][0].keys()

dict_keys(['flamevall', 'monitorl'])

In [12]:
isac_data['reconst_input']['reconst_input'].keys()

dict_keys(['model_engine', 'version', 'lattice_model', 'selem', 'eelem', 'flat', 'opt_method', 'iteration_no', 'comment', 'scan_type', 'monitorl', 'opt_flg', 'opt_target', 'measurement_select', 'moment_init_param', 'moment_weightl', 'moment_init_paraml', 'moment_weightl2', 'target_knobs_comment', 'target_knobs', 'monitor_knobs'])

In [13]:
assert isac_data['reconst_input']['reconst_input']['opt_target'][-1] == 'moment1'
isac_data['reconst_input']['reconst_input']['opt_target']

['moment1', 'moment1']

In [14]:
isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['yrms']

[2, 0, 2, 0, 2, 2]

In [15]:
isac_data['reconst_input']['reconst_input']['target_knobs_comment']

"((ename,[min_val, max_val, initial_val, field, unit]),('FE_LEBT:QHE_D0844',[-10, 10, 0, 'dy','mm']))"

# reconst output data

In [16]:
isac_data['reconst_output'].keys()

dict_keys(['meas', 'reconst_input', 'jsonfname', 'fjson_output', 'reconst_output'])

In [17]:
isac_data['reconst_output']['meas'][0].keys()

dict_keys(['flamevall', 'monitorl'])

In [18]:
isac_data['reconst_output']['reconst_input'].keys()

dict_keys(['model_engine', 'version', 'lattice_model', 'selem', 'eelem', 'flat', 'opt_method', 'iteration_no', 'comment', 'scan_type', 'monitorl', 'opt_flg', 'opt_target', 'measurement_select', 'moment_init_param', 'moment_weightl', 'moment_init_paraml', 'moment_weightl2', 'target_knobs_comment', 'target_knobs', 'monitor_knobs'])

In [19]:
isac_data['reconst_output']['reconst_output'].keys()

dict_keys(['posl', 'xdatal', 'xlabel', 'flat', 'pv_name', 'title', 'xcenl_meas', 'ycenl_meas', 'xcenl_sim', 'ycenl_sim', 'xrmsl_meas', 'yrmsl_meas', 'cxyl_meas', 'xrmsl_sim', 'yrmsl_sim', 'cxyl_sim', 'moment_init_param', 'moment_opt_param'])

In [20]:
isac_data['reconst_output']['reconst_output']['flat']

'/files/shared/ap/ISAAC/data/20231219_010647_maruta_FS1_CSS_PM_D2225_pv_scan/summary_20231219_010647_reconst_output_3d/flame_reconst_input.lat'

In [21]:
isac_data['reconst_output']['reconst_output']['moment_opt_param']

[0.21461250656852565,
 2.721108852864159,
 4.803736266085451,
 0.08534704254162553,
 1.5695144262528864,
 6.25210487850898,
 -0.0038059748618568976,
 -0.37852805595134853,
 0.1381066788259228,
 -0.18650971053339538]

# Read PM raw data

In [22]:
print(isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['xrms'])
print(isac_data['reconst_input']['reconst_input']['measurement_select'][-1]['yrms'])

[2, 0, 2, 0, 2, 2]
[2, 0, 2, 0, 2, 2]


In [23]:
scan_data = isac_data['reconst_summary']['scan_data']
len(scan_data)

6

In [24]:
raw_data_flist = [scan_data[i]['res_monitorl'][0]['file'][3:] for i in range(len(scan_data))]
raw_data_flist

['FS1_CSS_PM_D2225_20231219_010705.dat',
 'FS1_CSS_PM_D2225_20231219_012700.dat',
 'FS1_CSS_PM_D2225_20231219_010956.dat',
 'FS1_CSS_PM_D2225_20231219_011945.dat',
 'FS1_CSS_PM_D2225_20231219_013649.dat',
 'FS1_CSS_PM_D2225_20231219_012235.dat']

In [25]:
raw= pp.read_raw_data_from_directory(os.path.join(ISAAC_database_path,ISAAC_data_rel_path))
for rel_path in ih.get_related_ISAAC_data_rel_path(ISAAC_data_rel_path,ISAAC_database_path,within_minutes=100):
    raw.update(pp.read_raw_data_from_directory(os.path.join(ISAAC_database_path,rel_path)))
raw_data_flist2 = list(raw.keys())
raw_data_flist2

['FS1_CSS_PM_D2225_20231219_010705.dat',
 'FS1_CSS_PM_D2225_20231219_010956.dat',
 'FS1_CSS_PM_D2225_20231219_011945.dat',
 'FS1_CSS_PM_D2225_20231219_012235.dat',
 'FS1_CSS_PM_D2225_20231219_012700.dat',
 'FS1_CSS_PM_D2225_20231219_012951.dat',
 'FS1_CSS_PM_D2225_20231219_013649.dat']

In [26]:

for key, value in raw.items():
    r = 0.5 * value['wire_diam']
    value['postprocess'] = {'denoised': [],
                            'projected': {}}
    rms = []
    arr_rms = []
    for ic, coord in enumerate(value['coord'][1:]):
        x = value['lraw'][ic][0]
        y = value['lraw'][ic][1]
        with capture_warnings():
            tmp = sp.process_profile_signal(x, y, r)
        value['postprocess']['denoised'].append(tmp)
        rms.append(tmp['rms_deconv'])
        arr_rms.append(tmp['MC_stat']['arr_deconv_rms'])
    value.pop('lraw')

    with capture_warnings():
        x,y,cxy,u,v = sp.project_L3_to_xyuv(*rms,value)
    with capture_warnings():    
        arr_x,arr_y,arr_cxy,arr_u,arr_v = sp.project_L3_to_xyuv(*arr_rms,value)
    value['postprocess']['projected']['xrms']=x
    value['postprocess']['projected']['yrms']=y
    value['postprocess']['projected']['cxy']=cxy
    value['postprocess']['projected']['urms']=u
    value['postprocess']['projected']['vrms']=v
    value['postprocess']['projected']['xrms_err']=np.std(arr_x)
    value['postprocess']['projected']['yrms_err']=np.std(arr_y)
    value['postprocess']['projected']['cxy_err']=np.std(arr_cxy)
    value['postprocess']['projected']['u_err']=np.std(arr_u)
    value['postprocess']['projected']['v_err']=np.std(arr_v)

Warning message: invalid value encountered in sqrt
Warning type: RuntimeWarning
Originated from: /media/sf_HWANG/My Drive/GitHub/PMpostprocess/example/../PMpostprocess/signal_processing.py, line 201
Warning message: invalid value encountered in sqrt
Warning type: RuntimeWarning
Originated from: /media/sf_HWANG/My Drive/GitHub/PMpostprocess/example/../PMpostprocess/signal_processing.py, line 201
Warning message: invalid value encountered in sqrt
Warning type: RuntimeWarning
Originated from: /media/sf_HWANG/My Drive/GitHub/PMpostprocess/example/../PMpostprocess/signal_processing.py, line 201
Warning message: invalid value encountered in sqrt
Warning type: RuntimeWarning
Originated from: /media/sf_HWANG/My Drive/GitHub/PMpostprocess/example/../PMpostprocess/signal_processing.py, line 201
Warning message: invalid value encountered in sqrt
Warning type: RuntimeWarning
Originated from: /media/sf_HWANG/My Drive/GitHub/PMpostprocess/example/../PMpostprocess/signal_processing.py, line 201
Warni

In [27]:
# f = 'FS1_CSS_PM_D2225_20240528_200148.dat'
# plt.figure(figsize=(4,2))
# plt.plot(raw[f]['postprocess']['denoised'][0]['smoothed'])

# Organize for flame reconst input

In [28]:
from FRIB_model import flame_helper as fh
fm_orig = fh.ModelFlame(isac_data['fmlatfile'])
fm = fh.ModelFlame(isac_data['fmlatfile'])

In [29]:
nscan = len(isac_data['reconst_input']['meas'])
select = isac_data['reconst_input']['reconst_input']['measurement_select'][-1]
assert nscan == len(raw_data_flist)

fm_evals = {}
fm_goals = {}
fm_goals_postprocessed = {}
fm_goals_err = {}

for i in range(nscan):
    for fm_elem,val_field in isac_data['reconst_input']['meas'][i]['flamevall'].items():
        k = (fm_elem,val_field[1])
        if k not in fm_evals:
            fm_evals[k] = [None]*nscan
        fm_evals[k][i] = val_field[0]
        
    for fm_elem,meas in isac_data['reconst_input']['meas'][i]['monitorl'].items():
        for goal in ['xrms','yrms','cxy']:
            k = (fm_elem,goal)
            if k not in fm_goals:
                fm_goals[k] = [None]*nscan
                fm_goals_postprocessed[k] = [None]*nscan
                fm_goals_err[k] = [None]*nscan
            # if selected, record goal
            if select[goal][i]: 
                fm_goals[k][i] = meas[goal]
                fm_goals_postprocessed[k][i] = raw[raw_data_flist[i]]['postprocess']['projected'][goal]
                fm_goals_err[k][i] = raw[raw_data_flist[i]]['postprocess']['projected'][goal+'_err']
                
fm_evals = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_evals))
fm_goals = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_goals))
fm_goals_postprocessed = fh.make_FLAME_evals_or_goals(fm,df=pd.DataFrame(fm_goals_postprocessed))

fm_goals_err = pd.DataFrame(fm_goals_err)
goal_std = fm_goals_postprocessed['df'].std()
goal_err_mean = fm_goals_err.mean()

scaler = np.nanmean(goal_std/(goal_err_mean + 1e-2*goal_std))
fm_goals_postprocessed['err'] = fm_goals_err
fm_goals_postprocessed['normalization_factor'] = fm_goals_err*scaler

In [30]:
fm_evals

{'info': {'FS1_CSS:QH_D2194': {'index': 100},
  'FS1_CSS:QV_D2202': {'index': 104}},
 'df':   FS1_CSS:QH_D2194 FS1_CSS:QV_D2202
                 B2               B2
 0          5.79490        -18.89210
 1         10.76299        -18.89305
 2          9.11068        -18.89210
 3         12.42847        -18.89210
 4         14.80169        -18.89305
 5         15.75006        -18.89305}

In [31]:
fm_goals['df']

FS1_CSS:PM_D2225                  
              xrms     yrms      cxy
0          5.84524  3.22642 -0.41015
1              NaN      NaN -0.27948
2          3.37396  1.43202      NaN
3              NaN      NaN      NaN
4          0.59793  0.84524      NaN
5          1.06284  1.39575  0.83505

In [32]:
fm_goals_postprocessed['df']

FS1_CSS:PM_D2225                    
              xrms      yrms       cxy
0         5.844476  3.245183 -0.415817
1              NaN       NaN -0.547172
2         3.436656  1.436222 -0.999000
3              NaN       NaN       NaN
4         0.471386  0.943101  0.999000
5         1.035305  1.473927  0.746214

In [33]:
fm_goals_err

FS1_CSS:PM_D2225                        
              xrms      yrms           cxy
0         0.031652  0.020925  1.329134e-02
1              NaN       NaN  2.412499e-02
2         0.012574  0.009683  1.110223e-16
3              NaN       NaN           NaN
4         0.009603  0.030241           NaN
5         0.012093  0.019615  3.668430e-02

### compare fittings

In [34]:
fh.fit_moment1(fm,fm_evals,fm_goals_postprocessed,stop_criteria=0.1)
fm.bmstate.get_twiss('x'), fm.bmstate.xemittance, fm.bmstate.get_twiss('y'), fm.bmstate.yemittance

0-th trial, current_loss: 0.192, best_loss: 0.192
1-th trial, current_loss: 0.192, best_loss: 0.192
2-th trial, current_loss: 0.292, best_loss: 0.192
3-th trial, current_loss: 0.567, best_loss: 0.192
4-th trial, current_loss: 0.262, best_loss: 0.192


(array([-0.40701245, 12.95053029,  0.0900086 ]),
 1.280644931802985,
 array([0.96534406, 3.54496669, 0.5449668 ]),
 1.7133241256063994)

In [35]:
# fm_goals['normalization_factor'] = fm_goals_postprocessed['normalization_factor']
fh.fit_moment1(fm,fm_evals,fm_goals,stop_criteria=0.1)
fm.bmstate.get_twiss('x'), fm.bmstate.xnemittance, fm.bmstate.get_twiss('y'), fm.bmstate.ynemittance

0-th trial, current_loss: 0.021, best_loss: 0.021
2-th trial, current_loss: 0.0205, best_loss: 0.0205


(array([-0.72752939, 13.94254953,  0.10968575]),
 0.2635546253020422,
 array([1.3115474 , 0.94823549, 2.8686509 ]),
 0.048787186621729255)

In [36]:
fm_orig.bmstate.get_twiss('x'), fm_orig.bmstate.xnemittance, fm_orig.bmstate.get_twiss('y'), fm_orig.bmstate.ynemittance

(array([-0.43126043, 11.65162811,  0.10178711]),
 0.24018614529725893,
 array([0.95776394, 3.41514823, 0.56141392]),
 0.31260437450834555)